# Heuristic Branch and Bound for the Quadratic Assignment Problem

In [2]:
#Initialisation
import numpy as np
import pandas as pd
from itertools import product
import math

The functions below are for inputting the csv data (that is in the QAPLIB format) and making the matrices numpy arrays.

In [3]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    
    Optimised 04/06/2020
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = np.zeros(column)
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = np.array([])
            for i in string2:
                if i != '':
                    emptyarray = np.append(emptyarray,i).astype(int)
            matrix = np.vstack((matrix,emptyarray))
        return matrix[1:]
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [10]:
#small sized matrices(under 10x10) (quick on all methods)
matrix_size_4 = './data/made4.csv'
matrix_size_5 = './data/made5.csv'
matrix_size_6 = './data/made6.csv'
matrix_size_7 = './data/made7.csv'
matrix_size_8 = './data/made8.csv'
matrix_size_9 = './data/made9.csv'

matrixMade = ['./data/made4.csv', 
              './data/made5.csv', 
              './data/made6.csv', 
              './data/made7.csv', 
              './data/made8.csv', 
              './data/made9.csv']


#medium sized matrices(ranging from 10x10 to 30x30) (slow on deterministic methods, fast on heuristics)
matrix_size_10 = './data/tai10a.csv'
matrix_size_11 = './data/made11.csv'
matrix_size_12 = './data/tai12a.csv'
matrix_size_15 = './data/chr15a.csv' 
matrix_size_20 = './data/chr20a.csv'
matrix_size_26 = './data/bur26a.csv'

#large sized matrices(30x30 and bigger)(reasonably slow on the heuristics to a certain degree of accuracy)
matrix_size_40 = './data/tai40a.csv'
matrix_size_60 = './data/tai60.csv'
matrix_size_80 = './data/tai80.csv'
matrix_size_256 = './data/tai256c.csv'

datamatrix = CSVtoNumpyArray(matrix_size_4) # Decide the size of problem to run in the code (clue: 
                                                #the number in the original name is the size)
MatrixLoc = datamatrix[0]
MatrixFlow = datamatrix[1]

FileNotFoundError: [Errno 2] File b'./data/made4.csv' does not exist: b'./data/made4.csv'

In [4]:
datamatrix = CSVtoNumpyArray('made7.csv')
MatrixLoc = datamatrix[0]
MatrixFlow = datamatrix[1]

Below is the permutation generating function. For medium and large instances it would be wise to run this function to file separately first and then use the results for Exhaustive Search and/or Branch and Bound.

In [5]:
def QAPPermutations(iterable, r=None):
    """
    Input:
    String or numbers separated by a space
    optional= the length that the permutations must be
    
    Output:
    a generator of permutations
    
    Sourced from the itertools library and adjusted to suit this data type
    https://docs.python.org/3.4/library/itertools.html#itertools.permutations 
    """
    
    pool = iterable.split(" ")
    n = len(pool)
    r = n if r is None else r
    for indices in product(range(n), repeat=r):
        if len(set(indices)) == r:
            yield list(pool[i] for i in indices)

In [6]:
def HeuristicBranchandBound(zetta,iterations,MatrixLoc,MatrixFlow):
    """
    Input:
    zetta - probability between 0 and 1 - gauge of randomness
    iterations - int
    MatrixLoc
    MatrixFlow
    
    Output:
    the optimal cost
    The optimal permutation
    
    in a tuple
    """
    #initialization
    thestring = ""
    matrix_length = len(MatrixLoc)
    bettersol = math.inf

    #Generate the permutations
    for i in range(0,matrix_length):  #this is making a string of numbers from 0 to the size of the matrix -1
        thestring += str(i) + " "
    thestring = thestring[:-1]
    baselistofpermutations = np.array(list(QAPPermutations(thestring,2))).astype(int)
    
    for it in range(iterations):
        listofpermutations = baselistofpermutations
        t = 0
        while t < matrix_length-1:
            fin = np.array([])
            for perm in listofpermutations:
                
                #Cost-calculation
                l = len(perm)
                total = 0
                i = 0
                while i < l and total<bettersol:
                    j=0
                    while j < l and total<bettersol:
                        total += MatrixLoc[i][j]*MatrixFlow[perm[i]][perm[j]]
                        j += 1
                    i += 1
                fin = np.append(fin,total)
                
            #zetta is a probability - higher means more random
            if zetta>np.random.random():
                index = np.random.choice(list(range(len(fin))))
            else:
                index = 0
            begin = listofpermutations[np.argsort(fin)[index]]
            
            #Next permutations
            new = np.zeros(len(perm)+1)
            for v in range(matrix_length):
                if v not in begin:
                    begin = np.array(begin)
                    nbegin = np.append(begin,v)
                    new = np.vstack((new,nbegin))
            listofpermutations = new[1:].astype(int)
            if total>=bettersol:
                t = math.inf
            else:
                t += 1
        
        #Store best value
        if fin[np.argsort(fin)[index]]<bettersol and len(begin) == matrix_length:
            bettersol = fin[np.argsort(fin)[index]]
            bettersolallocation = begin
            
    return bettersol,bettersolallocation


In [7]:
HeuristicBranchandBound(0.8,40,MatrixLoc,MatrixFlow)

(1804.0, array([1, 2, 0, 5, 4, 3, 6]))

In [8]:
def gridsearch(names):
    np.random.seed(30)
    count = 1
    for q in range(len(names)): 
        datamatrix = CSVtoNumpyArray(names[q])
        MatrixLoc = datamatrix[0]
        MatrixFlow = datamatrix[1]
        itsizes = list(range(10,30*count,1*count))
        zsizes = np.linspace(0.1, 0.9, num=10)
        count += 1
        file = open("HBNB - out" + names[q] ,"w")
        for it in itsizes:
            for z in zsizes:#PACKSIZE
                sol = (HeuristicBranchandBound(z,it,MatrixLoc,MatrixFlow),z,it)
                file.write( str(sol) + "\n")
        file.close()
        print(names[q] + "has completed its search")
    return True